In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/mamabot-file/mamabot_data.csv')

df.dropna(inplace = True)

In [3]:
df.sample(10)

,Question,Answer
514,Why is protein important during pregnancy?,Protein is essential for the growth and develo...
257,What is “pregnancy brain”?,Pregnancy brain is a foggy feeling caused by p...
663,How can mothers ensure they are eating well-ba...,Mothers can ensure well-balanced meals by incl...
657,How can mothers prepare for labor in the final...,Mothers can prepare for labor by discussing bi...
285,How can mothers manage breast growth discomfor...,Mothers can manage breast growth discomfort by...
98,What are the benefits of prenatal yoga during ...,"Prenatal yoga helps improve flexibility, stren..."
115,What role does calcium play during pregnancy?,Calcium is essential for the development of th...
125,How is the baby's heart activity changing in w...,Fetal cardiac activity is becoming more regular.
112,How can mothers deal with unusual symptoms lik...,Maintaining good oral hygiene and consulting w...
93,What are the exercise recommendations during w...,Do not exercise on your back. Try not to hold ...


In [4]:
df.shape

(705, 2)

In [5]:
# Install necessary libraries
!pip install -q datasets huggingface_hub
!pip install -q -U peft
!pip install -q -U trl
!pip install -q git+https://github.com/huggingface/accelerate.git
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q bitsandbytes
!pip install -q wandb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.


In [6]:
# Import libraries
from datasets import Dataset, load_dataset
import pandas as pd
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,Trainer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,AutoConfig,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os
import time
import torch
from datasets import Dataset
from huggingface_hub import notebook_login, HfFolder
from trl import SFTTrainer,setup_chat_format, SFTConfig
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

2024-08-13 17:03:11.077293: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 17:03:11.077392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 17:03:11.197623: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
from huggingface_hub import HfApi
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token_3")  


# Set the Hugging Face API token as an environment variable
os.environ["HF_HOME"] = "/root/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/root/.cache/huggingface/transformers"
os.environ["HF_DATASETS_CACHE"] = "/root/.cache/huggingface/datasets"
os.environ["HF_METRICS_CACHE"] = "/root/.cache/huggingface/metrics"
os.environ["HF_HUB_TOKEN"] = hf_token

os.environ["HF_HUB_TOKEN"] = hf_token

In [8]:
wandb_key = user_secrets.get_secret("wandbi_api_2")
import wandb
! wandb login $wandb_key

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
# Define the location and configuration for the model
model_id = "/kaggle/input/llama-3.1/transformers/8b/1"

In [10]:
compute_dtype = torch.bfloat16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False
)


In [12]:
# Measure time for loading model and tokenizer
time_start = time.time()

# Load the model with quantization settings
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
    attn_implementation='eager'
)


# Measure and display time taken to load
time_end = time.time()
print(f"Prepare model, tokenizer: {round(time_end - time_start, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Prepare model, tokenizer: 101.377 sec.


In [13]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id,   trust_remote_code=True)

# Set up the chat format
model, tokenizer = setup_chat_format(model, tokenizer)

In [14]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

model.add_adapter(peft_config)
     

In [16]:
# Load the dataset
dataset = Dataset.from_pandas(df, split="all")

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Question"]},
               {"role": "assistant", "content": row["Answer"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][3]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/705 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


"<|im_start|>user\nWhat is HelpMum Africa's plan for mothers during their pregnancy journey?<|im_end|>\n<|im_start|>assistant\nHelpMum Africa plans to support mothers throughout their pregnancy journey, from the beginning until the baby is born.  <|im_end|>\n"

In [17]:
dataset['text'][5]

'<|im_start|>user\nWhen is it possible to confirm pregnancy with a test?<|im_end|>\n<|im_start|>assistant\nIt will take a few more weeks before pregnancy can be confirmed with a pregnancy test. <|im_end|>\n'

In [18]:
dataset = dataset.train_test_split(test_size=0.1)

In [19]:
os.environ["WANDB_DISABLED"] = "false"

In [20]:
from huggingface_hub import HfApi, HfFolder

# Set the token
HfFolder.save_token(hf_token)  

api = HfApi()
whoami = api.whoami(token=hf_token)
print(f"Logged in as: {whoami['name']}")

Logged in as: HelpMum-Personal


In [21]:
YOUR_HF_USERNAME = "Helpmum-Personal"

output_dir = f"{YOUR_HF_USERNAME}/Helpmum-Personal/mamabot-llama-adapter"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb",
    push_to_hub=True
)

# Create the Trainer without deprecated arguments
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1526: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1526: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1526: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Trans

Map:   0%|          | 0/634 [00:00<?, ? examples/s]

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

In [22]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: ajisco. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
191,0.982000,0.745381
382,0.431100,0.544463
573,0.692500,0.501459
764,0.202600,0.495271


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=951, training_loss=0.4841249056246629, metrics={'train_runtime': 1976.5661, 'train_samples_per_second': 0.962, 'train_steps_per_second': 0.481, 'total_flos': 4457305658916864.0, 'train_loss': 0.4841249056246629, 'epoch': 3.0})

In [23]:
# Evaluate the model
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.4952855110168457, 'eval_runtime': 30.6371, 'eval_samples_per_second': 2.317, 'eval_steps_per_second': 2.317, 'epoch': 3.0}


In [24]:
wandb.finish()
model.config.use_cache = True

eval/loss,█▂▁▁▁
eval/runtime,▄▁▅██
eval/samples_per_second,▆█▃▁▁
eval/steps_per_second,▆█▃▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▅▃▃▃▃▃▃▃▄▃▃▃▁▂▃▁▂▁▂▁▂▁▂▁▂▁▁▂▁▂▂▂▂▂▁▁▁▁▁
train/learning_rate,▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▅▅▅▃▄▃▅▅▅▆▂▆▂▁▂▁▄▁▃▁▃▁▅▁▂▁▁▃▁▂▂▃▂▂▂▁▂▁▁
eval/loss,0.49529
eval/runtime,30.6371
